

```
# First We will Install Tensorflow object detection COCO model, if it is installed already. Please skip this step.
```



In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib
!git clone https://github.com/tensorflow/models.git
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI 
!make
%cp -r pycocotools /content/models/research/
%cd /content/models/research
!mkdir train eval
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim
!protoc object_detection/protos/*.proto --python_out=.
!python object_detection/builders/model_builder_test.py

#If it is successfully installed, the last command will run otherwise it won't.

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.16-2~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 16 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,688 kB of additional disk space will be used.
Get:1 htt

In [0]:
%cd /content/
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import cv2
!unzip -q "drive/My Drive/football_Ball_Data.zip"

soccer_ball_data.zip  yolo_custom-2300.zip
annotations  generate_tfrecord.py  models	    sample_video
cocoapi      images		   pipeline.config
drive	     label_map.pbtxt	   sample_data


In [0]:
import os
allImages = os.listdir('images')
#Decrease Image size for faster processing
for i in range(len(allImages)):
    image = cv2.imread('images/'+allImages[i])
    image = cv2.resize(image, (800, 454), interpolation = cv2.INTER_AREA) 
    cv2.imwrite('images/'+allImages[i],image)

In [0]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [0]:
## XML TO CSV
xml_df = xml_to_csv('annotations/')
xml_df.to_csv(('train_labels.csv'), index=None)
print('Successfully converted xml to csv.')

Successfully converted xml to csv.


In [0]:
import numpy as np
traincsv = pd.read_csv('train_labels.csv')

#Change the annotations after changing the image size. If you resize the image, you also need to change it accordingly 
traincsv['width'] = 454
traincsv['height'] = 800
traincsv['xmin'] = np.round(traincsv['xmin']/2.40088105727).astype(int)
traincsv['ymin'] = np.round(traincsv['ymin']/2.4).astype(int)
traincsv['xmax'] = np.round(traincsv['xmax']/2.40088105727).astype(int)
traincsv['ymax'] = np.round(traincsv['ymax']/2.4).astype(int)

In [0]:
#Split the dataset into train & test set and save into csv file
traincsv = traincsv.sample(random_state=42,n=traincsv.shape[0])
train = traincsv.iloc[:211,:]
test = traincsv.iloc[211:,:]
train.to_csv(('train_labels.csv'), index=None)
test.to_csv(('test_labels.csv'), index=None)

In [0]:
#Generate python .record file
!python generate_tfrecord.py --csv_input=train_labels.csv --image_dir=images --output_path=train.record
!python generate_tfrecord.py --csv_input=test_labels.csv --image_dir=images --output_path=test.record

W0703 05:47:04.995344 139754920171392 deprecation_wrapper.py:119] From generate_tfrecord.py:102: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0703 05:47:04.995892 139754920171392 deprecation_wrapper.py:119] From generate_tfrecord.py:88: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W0703 05:47:05.076368 139754920171392 deprecation_wrapper.py:119] From generate_tfrecord.py:47: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/train.record
W0703 05:47:09.047677 140582018344832 deprecation_wrapper.py:119] From generate_tfrecord.py:102: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0703 05:47:09.048374 140582018344832 deprecation_wrapper.py:119] From generate_tfrecord.py:88: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W0703 05:47:09.065510 140582018344832 de

In [0]:
#DOwnload Faster_rnn_inception model
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

--2019-07-03 05:47:14--  http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.124.128, 2607:f8b0:4001:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.124.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149119618 (142M) [application/x-tar]
Saving to: ‘faster_rcnn_inception_v2_coco_2018_01_28.tar.gz’

faster_rcnn_incepti 100%[===================>] 142.21M  93.1MB/s    in 1.5s    

2019-07-03 05:47:16 (93.1 MB/s) - ‘faster_rcnn_inception_v2_coco_2018_01_28.tar.gz’ saved [149119618/149119618]



In [0]:
!tar -zxvf 'faster_rcnn_inception_v2_coco_2018_01_28.tar.gz'

faster_rcnn_inception_v2_coco_2018_01_28/
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.index
faster_rcnn_inception_v2_coco_2018_01_28/checkpoint
faster_rcnn_inception_v2_coco_2018_01_28/pipeline.config
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.meta
faster_rcnn_inception_v2_coco_2018_01_28/saved_model/
faster_rcnn_inception_v2_coco_2018_01_28/saved_model/saved_model.pb
faster_rcnn_inception_v2_coco_2018_01_28/saved_model/variables/
faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb


In [0]:
#Colab was throwing some error, so I added this line to fix it.
!pip install 'prompt-toolkit==1.0.15'

     |████████████████████████████████| 256kB 3.4MB/s 
ERROR: jupyter-console 6.0.0 has requirement prompt-toolkit<2.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.15 which is incompatible.
  Found existing installation: prompt-toolkit 2.0.9
    Uninstalling prompt-toolkit-2.0.9:
      Successfully uninstalled prompt-toolkit-2.0.9


In [0]:
import shutil
shutil.move("faster_rcnn_inception_v2_coco_2018_01_28", "models/research/object_detection")

'models/research/object_detection/faster_rcnn_inception_v2_coco_2018_01_28'

In [0]:
shutil.move("train.record", "models/research/object_detection")

'models/research/object_detection/train.record'

In [0]:
shutil.move("test.record", "models/research/object_detection")

'models/research/object_detection/test.record'

In [0]:
!mkdir models/research/object_detection/training
shutil.move("pipeline.config", "models/research/object_detection/training")

'models/research/object_detection/training/pipeline.config'

In [0]:
shutil.move("label_map.pbtxt", "models/research/object_detection/training")

'models/research/object_detection/training/label_map.pbtxt'

In [0]:
%cd /content/models/research/object_detection

/content/models/research/object_detection


In [0]:

#Train the model. It is supposed to save the trained checkpoints in training folder. But it is saving them in tmp folder. 
#After it saves a checkpoint, you need to note down the folder name inside tmp, where the checkpoint was saved. You can find it in output
!python3 model_main.py --logtostderr --train_dir=training/ --pipeline_config_path=training/pipeline.config

W0703 05:47:35.581923 139918654441344 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0703 05:47:35.616981 139918654441344 deprecation_wrapper.py:119] From /content/models/research/slim/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0703 05:47:35.627609 139918654441344 deprecation_wrapper.py:119] From /content/models/research/slim/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0703 05:47:35.674051 139918654441344 deprecation_wrapper.py:119] From model_main.py:109: The name tf.app.run is deprecated. Please use tf.compat.v1.app

In [0]:
%cd object_detection

[Errno 2] No such file or directory: 'object_detection'
/content/models/research/object_detection


In [0]:
#MAKE SURE TO CHANGE THE FOLDER NAME HERE
!ls /tmp/CHANGE THIS FOLDER NAME TO WHERE THE FOLDER WAS SAVED/

ls: cannot access '/tmp/tmp3dg9clxo/': No such file or directory


In [0]:
#CHANGE THIS FOLDER NAME TO WHERE THE FOLDER WAS SAVED
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/pipeline.config --trained_checkpoint_prefix /tmp/CHANGE THIS FOLDER NAME TO WHERE THE FOLDER WAS SAVED/model.ckpt-20891 --output_directory inference_graph

W0703 06:30:08.530471 139754550966144 deprecation_wrapper.py:119] From /content/models/research/slim/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0703 06:30:08.541352 139754550966144 deprecation_wrapper.py:119] From /content/models/research/slim/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0703 06:30:08.552000 139754550966144 deprecation_wrapper.py:119] From export_inference_graph.py:156: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0703 06:30:08.552611 139754550966144 deprecation_wrapper.py:119] From export_inference_graph.py:139: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0703 06:30:08.558423 139754550966144 deprecation_wrapper.py:119] From /content/models/research/object_detection/exporter.py:367: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

W

TRAINING Process Is completed. Now Let's test the model

In [0]:
%cd ../
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import label_map_util

/content/models/research


In [0]:
%cd /content/models/research/object_detection

/content/models/research/object_detection


In [0]:
#Inference code for detecting the object and making a green circle around it

import numpy as np
from PIL import Image

def load_image_into_numpy_array(image):
  im_height, im_width, depth = image.shape
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

import tensorflow as tf

def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict


import os
from matplotlib import pyplot as plt
IMAGE_SIZE = (4,3)
PATH_TO_LABELS = os.path.join('training', 'label_map.pbtxt')

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)



# What model to download.

# Path to frozen detection graph. This is the actual model that is used for the object detection.
#PATH_TO_FROZEN_GRAPH = 'faster_rcnn_inception_v2_coco_2018_01_28' + '/frozen_inference_graph.pb'

PATH_TO_FROZEN_GRAPH =  'inference_graph/frozen_inference_graph.pb'

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [0]:
%cd /content/

/content


In [0]:
!ls

In [0]:
#Detect On A Image using this code. I am detecting on all iamges inside "images" folder

import os
import time
allImages = os.listdir('images')
print(len(allImages))
start = time.time()
for i in range (len(allImages)):
    image = cv2.imread('images/'+allImages[i])
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    #plt.figure(figsize=IMAGE_SIZE)
    #print(image_np.shape)
    cv2.imwrite('predictions/'+allImages[i],image)

    break
print("Time taken", start-time.time())

In [0]:
#If you want to predict on a video, you need to convert all video frames to image then predict, then again to video.

import math
!mkdir videoFrames
count = 0
videoFile = "sample_video/test_video.mp4"
cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
frameRate = cap.get(5) #frame rate

while(cap.isOpened()):
    frameId = cap.get(25) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="videoFrames/frame%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
        
        
cap.release()
print ("Done!")

Done!


In [0]:
print(len(os.listdir('videoFrames')))

3601


In [0]:
import os
import time
!mkdir predictedFrames
allImages = os.listdir('videoFrames')
allImages.sort()
print(len(allImages))
start = time.time()
for i in range (len(allImages)):
    image = cv2.imread('videoFrames/'+allImages[i])
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    cv2.imwrite('predictedFrames/'+allImages[i],image)
    if i%100 == 0:
      print(i)
print("Time taken", start-time.time())

3601
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600


In [0]:
#Convert image to video with 25 fps
import os
from os.path import isfile, join
import cv2

def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
 
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
 
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)

    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

pathIn= "predictedFrames/"
pathOut = 'outputVideo2.avi'
fps = 25 #CHANGE FPS HERE
convert_frames_to_video(pathIn, pathOut, fps)

In [0]:
print("DONE")